In [15]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 11592



### data

In [22]:
sas.submitLST(f"""
    data cars;
        set sashelp.cars;
        if strip(model) in ('TSX 4dr', 'TL 4dr', 'MDX') then do;
            msrp = .;
            invoice = .;
            mpg_city = .;
            mpg_highway = .;
        end;
        if strip(model) in ('3.5 RL 4dr') then do;
            type = '';
            origin = '';
        end;
        if msrp > 32762 then expensive = 1;
        else expensive = 0;
        keep model type origin msrp invoice mpg_city mpg_highway expensive;
    run;
              
    proc sort data=cars nodupkey;
        by model;
    run;

    proc print data=cars (obs=5);
    run;
""")

Obs,Model,Type,Origin,MSRP,Invoice,MPG_City,MPG_Highway,expensive
1,3.5 RL 4dr,,,"$43,755","$39,014",18,24,1
2,3.5 RL w/Navigation 4dr,Sedan,Asia,"$46,100","$41,100",18,24,1
3,300M 4dr,Sedan,USA,"$29,865","$27,797",18,27,0
4,300M Special Edition 4dr,Sedan,USA,"$33,295","$30,884",18,27,1
5,325Ci 2dr,Sedan,Europe,"$30,795","$28,245",20,29,0


### analysis

In [25]:
# basic
sas.submitLST(
f"""
proc means data=cars;
    var msrp;
run;
""")

In [39]:
# proc summary
# stmnts: by, output
sas.submitLST(
f"""
proc sort data=cars;
    by type;

proc summary data=cars;
    var expensive;
    by type;
    output out=_means mean=avg_expensive;
run;

proc print data=_means;
run;
""")

Obs,Type,_TYPE_,_FREQ_,avg_expensive
1,,0,1,1.00000
2,Hybrid,0,3,0.00000
3,SUV,0,60,0.46667
4,Sedan,0,258,0.30233
5,Sports,0,49,0.69388
6,Truck,0,24,0.20833
7,Wagon,0,30,0.36667


In [43]:
# comprehensive summary
# options: maxdec missing
# statements: by, output
sas.submitLST(f"""
    PROC SORT DATA = cars OUT = _sorted_for_means;
    BY type;

    title;
    PROC summary DATA = _sorted_for_means MAXDEC = 2 MISSING;
        where mpg_city > 1;
        BY type; 
        VAR msrp expensive;
        output out = _means_out 
            mean(msrp expensive) = mean_msrp mean_expensive;
    RUN;
              
    PROC PRINT DATA = _means_out;
    RUN;
""")

Obs,Type,_TYPE_,_FREQ_,mean_msrp,mean_expensive
1,,0,1,"$43,755",1.00000
2,Hybrid,0,3,"$19,920",0.00000
3,SUV,0,59,"$34,754",0.47458
4,Sedan,0,256,"$29,656",0.30469
5,Sports,0,49,"$53,387",0.69388
6,Truck,0,24,"$24,941",0.20833
7,Wagon,0,30,"$28,841",0.36667


In [41]:
# class stmnt
# options: missing
# statements: by, class, output
sas.submitLST(f"""
    PROC SORT DATA = cars OUT = _sorted_for_means;
    BY type;

    PROC summary DATA = _sorted_for_means missing;
        BY type;
        class origin;
        VAR expensive;
        OUTPUT OUT = _means 
            N = n
            mean = avg_expensive
        ;
    
    PROC PRINT DATA = _means;
    RUN;
""")

Obs,Type,Origin,_TYPE_,_FREQ_,n,avg_expensive
1,,,0,1,1,1.00000
2,,,1,1,1,1.00000
3,Hybrid,,0,3,3,0.00000
4,Hybrid,Asia,1,3,3,0.00000
5,SUV,,0,60,60,0.46667
6,SUV,Asia,1,25,25,0.28000
7,SUV,Europe,1,10,10,0.90000
8,SUV,USA,1,25,25,0.48000
9,Sedan,,0,258,258,0.30233
10,Sedan,Asia,1,92,92,0.07609


In [42]:
# nway stmnt
# options: missing, nway
# statements: by, class, output
sas.submitLST(f"""
    PROC SORT DATA = cars OUT = _sorted_for_means;
    BY type;

    PROC summary DATA = _sorted_for_means missing nway;
        BY type;
        class origin;
        VAR expensive;
        OUTPUT OUT = _means 
            N = n
            mean = avg_expensive
        ;
    
    PROC PRINT DATA = _means;
    RUN;
""")

Obs,Type,Origin,_TYPE_,_FREQ_,n,avg_expensive
1,,,1,1,1,1.00000
2,Hybrid,Asia,1,3,3,0.00000
3,SUV,Asia,1,25,25,0.28000
4,SUV,Europe,1,10,10,0.90000
5,SUV,USA,1,25,25,0.48000
6,Sedan,Asia,1,92,92,0.07609
7,Sedan,Europe,1,76,76,0.72368
8,Sedan,USA,1,90,90,0.17778
9,Sports,Asia,1,17,17,0.23529
10,Sports,Europe,1,23,23,1.00000
